## **Import and load the data file**

In [1]:
import nltk
nltk.download('punkt')#Sentence tokenizer

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

# **Preprocessing**

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/kaggle/input/chatbot-dataset/intents.json').read() # read json file
intents = json.loads(data_file) # load json file

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)# add each elements into list
        #combination between patterns and intents
        documents.append((w, intent['tag']))#add single element into end of list
        # add to tag in our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
nltk.download('wordnet') #lexical database for the English language

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [8]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

405 documents
 [(['Hi'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), (['?', '?', '?', '?', '?', '?', '?', '?'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creato

# **Training Model**

In [9]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words
    pattern_words = doc[0]
    # convert pattern_words in lower case
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create bag of words array,if word match found in current pattern then put 1 otherwise 0.[row * colm(263)]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # in output array 0 value for each tag ang 1 value for matched tag.[row * colm(8)]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle training and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test. X - patterns(words), Y - intents(tags)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [10]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

# **Build the model** 

In [11]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer:",model.layers[0].get_weights()[0])

First layer: [[ 0.08108504 -0.06599443 -0.10388638 ... -0.01234975  0.02568085
   0.00633688]
 [-0.02540757 -0.0221673  -0.0489299  ...  0.10772091  0.00711305
   0.03869867]
 [-0.06639696 -0.05009066 -0.03959011 ... -0.0571945  -0.11444904
  -0.06228179]
 ...
 [ 0.02686372  0.0873628   0.12299983 ... -0.07360662  0.05407895
  -0.01691054]
 [-0.08417445 -0.10581411 -0.07542053 ... -0.06181952 -0.12180413
  -0.08388676]
 [-0.07259022  0.11421812 -0.04386763 ...  0.00979565  0.05784626
   0.09121044]]


In [12]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
81/81 [==============================] - 1s 2ms/step - loss: 3.6136 - accuracy: 0.0543
Epoch 2/200
81/81 [==============================] - 0s 2ms/step - loss: 3.4736 - accuracy: 0.1259
Epoch 3/200
81/81 [==============================] - 0s 2ms/step - loss: 3.2848 - accuracy: 0.1753
Epoch 4/200
81/81 [==============================] - 0s 2ms/step - loss: 3.0604 - accuracy: 0.2346
Epoch 5/200
81/81 [==============================] - 0s 2ms/step - loss: 2.8305 - accuracy: 0.2716
Epoch 6/200
81/81 [==============================] - 0s 2ms/step - loss: 2.5375 - accuracy: 0.3432
Epoch 7/200
81/81 [==============================] - 0s 2ms/step - loss: 2.3111 - accuracy: 0.4025
Epoch 8/200
81/81 [==============================] - 0s 2ms/step - loss: 2.1470 - accuracy: 0.4568
Epoch 9/200
81/81 [==============================] - 0s 2ms/step - loss: 1.9539 - accuracy: 0.4864
Epoch 10/200
81/81 [==============================] - 0s 2ms/step - loss: 1.7000 - accuracy: 0.6025
Epoch 11/

# FOR PREDICTING RESPONSE
### **You have to add or run this below script by loading model**

### **For loading saved model**
//
from keras.models import load_model

model = load_model('chatbot_model.h5')
//
### **Predict the response**
To predict the sentences and get a response from the user to let us create a new file ‘chatapp.py’.
- We will load the trained model and then use a graphical user interface that will predict the response from the bot. The model will only tell us the class it belongs to, so we will implement some functions which will identify the class and then retrieve us a random response from the list of responses.
- Again we import the necessary packages and load the ‘words.pkl’ and ‘classes.pkl’ pickle files which we have created when we trained our model.

//---

intents = json.loads(open('/kaggle/input/chatbot-dataset/intents.json').read())

words = pickle.load(open('words.pkl','rb'))

classes = pickle.load(open('classes.pkl','rb'))

//---

**To predict the class, we will need to provide input in the same way as we did while training. So we will create some functions that will perform text preprocessing and then predict the class**

//----

#Utility Methods

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    
    sentence_words = nltk.word_tokenize(sentence)
    #print(sentence_words)
    # stem each word - create short form for word
    
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    #print(sentence_words)
    
    return sentence_words
#return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    
    sentence_words = clean_up_sentence(sentence)
    #print(sentence_words)
    
    # bag of words - matrix of N words, vocabulary matrix
    
    bag = [0]*len(words) 
    #print(bag)
    
    for s in sentence_words:  
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
                #print ("found in bag: %s" % w)
    #print(bag)
    return(np.array(bag))
    
def predict_class(sentence, model):
    # filter out predictions below a threshold
    
    p = bow(sentence, words,show_details=False)
    #print(p)
    
    res = model.predict(np.array([p]))[0]
    #print(res)
    
    ERROR_THRESHOLD = 0.25
    
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    #print(results)
    # sort by strength of probability
    
    results.sort(key=lambda x: x[1], reverse=True)
    #print(results)
    
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    
    return return_list
    #print(return_list)

//----

**After predicting the class, we will get a random response from the list of intents:**

//----

def getResponse(ints, intents_json):
    
    tag = ints[0]['intent']
    #print(tag)
    
    list_of_intents = intents_json['intents']
    #print(list_of_intents)
    
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
    
def chatbot_response(text):
    ints = predict_class(text, model)
    #print(ints)
    
    res = getResponse(ints, intents)
    #print(res)
    return res
    
//---  
**Enter you queries**   
//----   
start = True

while start:

    query = input('Enter Message:')
    if query in ['quit','exit','bye']:
        start = False
        continue
    try:
        res = chatbot_response(query)
        print(res)
    except:
        print('You may need to rephrase your question.')
//-----